Using RandomForest First

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_data = pd.read_csv('data.csv')
raw_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X = raw_data.iloc[:,:-1].values
y = raw_data.iloc[:,-1].values
X.shape,y.shape

((768, 8), (768,))

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , roc_auc_score
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)
rf = RandomForestClassifier(n_estimators=200,random_state=42)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
y_pred_prob = rf.predict_proba(X_test)[:,1]
print(accuracy_score(y_test,y_pred))
print(roc_auc_score(y_test,y_pred_prob))

0.7395833333333334
0.8070578531872276


Using a Neural Network

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_norm = sc.fit_transform(X_train)
X_test_norm = sc.transform(X_test)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(12, input_dim=8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 12)                  │             108 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              13 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

In [13]:
from keras.optimizers import SGD
model.compile(SGD(learning_rate = 0.003),"binary_crossentropy", metrics=['accuracy'])
runhist = model.fit(X_train_norm,y_train,epochs=200,validation_data=(X_test_norm,y_test))

Epoch 1/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3899 - loss: 0.7371 - val_accuracy: 0.3698 - val_loss: 0.7366
Epoch 2/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3289 - loss: 0.7555 - val_accuracy: 0.3802 - val_loss: 0.7231
Epoch 3/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3946 - loss: 0.7116 - val_accuracy: 0.4010 - val_loss: 0.7110
Epoch 4/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3953 - loss: 0.7155 - val_accuracy: 0.4583 - val_loss: 0.7001
Epoch 5/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4948 - loss: 0.6977 - val_accuracy: 0.4948 - val_loss: 0.6903
Epoch 6/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5366 - loss: 0.6921 - val_accuracy: 0.5260 - val_loss: 0.6815
Epoch 7/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5376 - loss: 0.6835 - val_accuracy: 0.5781 - val_loss: 0.6736
Epoch 8/200
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5613 - loss: 0.6840 - val_accuracy: 0.6146 - 

In [15]:
y_pred_prob = model.predict(X_test_norm)
y_pred = np.where(y_pred_prob>0.5,1,0)
print(accuracy_score(y_test,y_pred))
print(roc_auc_score(y_test,y_pred_prob))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.7395833333333334
0.8145398845292801


Training 2 layer neural network


In [16]:
model2 = Sequential()
model2.add(Dense(6, input_dim=8, activation='relu'))
model2.add(Dense(6, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))


In [17]:
model2.compile(SGD(learning_rate = 0.003),"binary_crossentropy", metrics=['accuracy'])
model2.fit(X_train_norm,y_train,epochs=1500,validation_data=(X_test_norm,y_test))
y_pred_prob = model2.predict(X_test_norm)
y_pred = np.where(y_pred_prob>0.5,1,0)
print(accuracy_score(y_test,y_pred))

Epoch 1/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.3257 - loss: 0.8416 - val_accuracy: 0.3854 - val_loss: 0.8517
Epoch 2/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3483 - loss: 0.8111 - val_accuracy: 0.3906 - val_loss: 0.8331
Epoch 3/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3518 - loss: 0.8260 - val_accuracy: 0.3906 - val_loss: 0.8166
Epoch 4/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3847 - loss: 0.7988 - val_accuracy: 0.3854 - val_loss: 0.8020
Epoch 5/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4028 - loss: 0.7782 - val_accuracy: 0.3854 - val_loss: 0.7891
Epoch 6/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4076 - loss: 0.7514 - val_accuracy: 0.4010 - val_loss: 0.7775
Epoch 7/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4543 - loss: 0.7463 - val_accuracy: 0.4167 - val_loss: 0.7672
Epoch 8/1500
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4451 - loss: 0.7590 - val_accuracy: 0

In [18]:
print(accuracy_score(y_test,y_pred))
print(roc_auc_score(y_test,y_pred_prob))

0.71875
0.7740073052904443
